In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

In [2]:
model = Network('model')

node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[1].set_number_of_servers(2)

jobclass = np.empty(3, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 2, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 0, node[0], 0)
jobclass[2] = ClosedClass(model, 'Class3', 1, node[0], 0)

node[0].set_service(jobclass[0], Erlang(3, 2))
node[0].set_service(jobclass[1], HyperExp(0.5, 3.0, 10.0))
node[0].set_service(jobclass[2], Exp(1))

node[1].set_service(jobclass[0], HyperExp(0.1, 1.0, 10.0))
node[1].set_service(jobclass[1], Exp(2))
node[1].set_service(jobclass[2], Exp(3))

P = model.init_routing_matrix()

# P{1,1} = [0.3,0.1; 0.2,0]
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.3)
P.set(jobclass[0], jobclass[0], node[0], node[1], 0.1)
P.set(jobclass[0], jobclass[0], node[1], node[0], 0.2)

# P{1,2} = [0.6,0; 0.8,0]
P.set(jobclass[0], jobclass[1], node[0], node[0], 0.6)
P.set(jobclass[0], jobclass[1], node[1], node[0], 0.8)

# P{2,1} = [0,0; 1,0]
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)

# P{2,2} = [0,1; 0,0]
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

# P{3,3} = circul(M)
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)
P.set(jobclass[2], jobclass[2], node[1], node[0], 1.0)

model.link(P)

In [3]:
solver = np.array([], dtype=object)
solver = np.append(solver, SolverCTMC(model))
solver = np.append(solver, SolverJMT(model, seed=23000, samples=5000, verbose=False))
solver = np.append(solver, SolverSSA(model, seed=23000, samples=5000, verbose=False))
solver = np.append(solver, SolverFluid(model))
solver = np.append(solver, SolverMVA(model))
solver = np.append(solver, SolverNC(model, method='exact'))
solver = np.append(solver, SolverMAM(model))
solver = np.append(solver, LINE(model, seed=23000))

AvgTable = np.empty(len(solver), dtype=object)
AvgChainTable = np.empty(len(solver), dtype=object)
AvgSysTable = np.empty(len(solver), dtype=object)

for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    try:
        AvgTable[s] = solver[s].get_avg_table()
        AvgChainTable[s] = solver[s].getAvgChainTable()
        AvgSysTable[s] = solver[s].getAvgSysTable()
    except Exception as e:
        print(f'Error with {solver[s].get_name()}: {str(e)[:100]}')
        # Set to None for failed solvers
        AvgTable[s] = None
        AvgChainTable[s] = None  
        AvgSysTable[s] = None


SOLVER: SolverCTMC
CTMC analysis [method: default, lang: java, env: 21.0.8] completed in 0.078365s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  1.1235  1.1235  0.6667  0.3968  1.6853  1.6853
1   Delay   Class2  0.2483  0.2483  0.2167  0.0877  1.1460  1.1460
2   Delay   Class3  0.7413  0.7413  1.0000  1.0000  0.7413  0.7413
3  Queue1   Class1  0.0332  0.0160  0.1973  0.0117  0.1685  0.1685
4  Queue1   Class2  0.5949  0.2865  0.5191  0.2101  1.1460  1.1460
5  Queue1   Class3  0.2587  0.1236  0.3489  0.3489  0.7413  0.7413
  Station   Chain    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay  Chain1  1.3718  1.3718  0.4845  0.2717  2.8313  2.8313
1   Delay  Chain2  0.7413  0.7413  1.0000  1.0000  0.7413  0.7413
2  Queue1  Chain1  0.6282  0.3025  0.4779  0.1847  1.3145  1.3145
3  Queue1  Chain2  0.2587  0.1236  0.3489  0.3489  0.7413  0.7413
Error with SolverCTMC: java.lang.IllegalArgumentException: Outside of matrix bounds

SOLVER: SolverJ